In [1]:
using ModelingToolkit, OrdinaryDiffEq
using ModelingToolkit: t_nounits as t
using CoolProp.CoolProp
using CoolProp.CoolProp: PropsSI
using BenchmarkTools

In [2]:
# Helper functions
function calc_T(h, p, fluid_model)
    HP_inputs = get_input_pair_index("HmassP_INPUTS")
    AbstractState_update(Int32(fluid_model), HP_inputs, Real(h), Real(p))
    property_index = get_param_index("T")
    T = AbstractState_keyed_output(Int32(fluid_model), property_index)
    return T
end

@register_symbolic calc_T(h::Real, p::Real, fluid_model::Int32);

In [3]:
# Helper functions
function calc_fluid_state(h, p, fluid_model)
    HP_inputs = get_input_pair_index("HmassP_INPUTS")
    AbstractState_update(Int32(fluid_model), HP_inputs, Real(h), Real(p))
    return fluid_model
end

function calc_fluid_property(property::String, fluid_state)
    property_index = get_param_index(property)
    return AbstractState_keyed_output(Int32(fluid_state), property_index)
end

# Register with more specific types for symbolic computation
@register_symbolic calc_fluid_state(h::Real, p::Real, fluid_model::Int32)
@register_symbolic calc_fluid_property(property::String, fluid_state::Int32)

In [4]:
@connector FluidPort begin
    @parameters begin
        fluid_model
    end

    @variables begin
        h(t), [guess=0.0]
        p(t), [guess=0.0]
        ṁ(t), [guess=0.0, connect = Flow]
    end
end

@connector FluidProperties begin
    @parameters begin
        fluid_model
    end

    @variables begin
        ṁ(t), [guess=0.0, connect = Flow]
    end

    @equations begin
        ṁ ~ 0
    end
end

@mtkmodel Source begin
    @parameters begin
        h_source
        p_source
        ṁ_source
    end

    @components begin
        outlet = FluidPort()
    end

    @equations begin
        outlet.h ~ h_source
        outlet.p ~ p_source
        outlet.ṁ ~ ṁ_source
    end
end

@mtkmodel Sink begin
    @components begin
        inlet = FluidPort()
    end

    @variables begin
        T(t)
        state(t)
    end

    @equations begin
        state ~ calc_fluid_state(inlet.h, inlet.p, inlet.fluid_model)
        T ~ calc_fluid_property("T", state)
    end
end

@mtkmodel System begin
    @parameters begin
        h_source
        p_source
        ṁ_source
        fluid_model::Int32
    end

    @components begin
        properties = FluidProperties(; fluid_model=fluid_model)
        source = Source(; h_source=h_source, p_source=p_source, ṁ_source=ṁ_source)
        sink = Sink()
    end

    @variables begin
        T(t)
    end

    @equations begin
        connect(properties, source.outlet)
        connect(source.outlet, sink.inlet)
        T ~ sink.T
    end
end;

In [5]:
# 
fluid_name = "Hydrogen"
eos = "HEOS"
fluid_model = AbstractState_factory(eos, fluid_name)

# 
ṁ_source = 10e-3            # kg/s
T_source = 70               # K
p_source = 350e5           # Pa
h_source = PropsSI("H", "T", T_source, "P", p_source, fluid_name)
ṁ_source = 10e-3;

In [6]:
fluid_state = calc_fluid_state(h_source, p_source, fluid_model)

0

In [7]:
calc_fluid_property("D", fluid_state)

67.30013202474397

In [8]:
@named sys_ = System(; h_source=h_source, p_source=p_source, 
ṁ_source=ṁ_source, fluid_model=fluid_model)

┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\matth\.julia\packages\ModelingToolkit\aau6A\src\systems\connectors.jl:54
┌ Warning: inlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\matth\.julia\packages\ModelingToolkit\aau6A\src\systems\connectors.jl:54


Model sys_:
Subsystems (3): see hierarchy(sys_)
  properties
  source
  sink
Equations (10):
  7 standard: see equations(sys_)
  3 connecting: see equations(expand_connections(sys_))
Unknowns (10): see unknowns(sys_)
  T(t)
  properties₊ṁ(t)
  source₊outlet₊h(t)
  source₊outlet₊p(t)
  ⋮
Parameters (10): see parameters(sys_)
  h_source [defaults to 865483.0]
  p_source [defaults to 3.5e7]
  ṁ_source [defaults to 0.01]
  fluid_model [defaults to 0]
  ⋮

In [9]:
complete_ = complete(sys_)

Model sys_:
Equations (10):
  10 standard: see equations(sys_)
Unknowns (10): see unknowns(sys_)
  T(t)
  properties₊ṁ(t)
  source₊outlet₊h(t)
  source₊outlet₊p(t)
  ⋮
Parameters (10): see parameters(sys_)
  source₊h_source [defaults to h_source]
  source₊p_source [defaults to p_source]
  ṁ_source [defaults to 0.01]
  source₊outlet₊fluid_model [defaults to properties₊fluid_model]
  ⋮

In [10]:
sys = structural_simplify(sys_)

Model sys_:
Parameters (10): see parameters(sys_)
  source₊h_source [defaults to h_source]
  source₊p_source [defaults to p_source]
  ṁ_source [defaults to 0.01]
  source₊outlet₊fluid_model [defaults to properties₊fluid_model]
  ⋮
Observed (10): see observed(sys_)

In [11]:
prob = ODEProblem(sys, [], (0, 10.0));

In [12]:
sol = solve(prob, Tsit5())

retcode: Success
Interpolation: 1st order linear
t: 2-element Vector{Float64}:
  0.0
 10.0
u: 2-element Vector{Vector{Float64}}:
 []
 []

In [13]:
println(sol[sys.T][end])

69.99999999999423
